# TP4

Daniel Francisco Texeira Andrade - A100057

Pedro André Ferreira Malainho - A100050

----

# Problema 1
## Enunciado

----
No contexto do sistema de travagem ABS (“Anti-Lock Breaking System”), pretende-se construir um autómato híbrido que descreva o sistema e que  possa ser usado para verificar as suas propriedades dinâmicas.

    

A componente discreta do autómato contém os modos:  `Start`,  `Free`,  `Stopping`, `Blocked`, e `Stopped`. No modo `Free`  não existe qualquer força de travagem; no modo `Stopping` aplica-se a força de travagem alta; no modo `Blocked` as rodas estão bloqueadas em relação ao corpo mas o veículo  desloca-se; no modo `Stopped` o veículo está imobilizado.

A componente contínua  do autómato usa variáveis contínuas $\,V,v\,$ para descrever a  `velocidade do corpo`  do veículo em relação ao solo e a `velocidade linear das rodas` também em relação ao solo.  Assume-se que o sistema de travagem exerce uma força de atrito  nos travões proporcional à diferença das duas velocidades.  A dinâmica contínua está descrita  abaixo no bloco Equaçoes de Fluxo.

a) Defina um autómato híbrido que descreva a dinâmica do sistema segundo as notas abaixo indicadas e com os “switchs” por si escolhidos. Os “switchs” (“jumps”) são uma  componente de projeto deste trabalho; cabe ao aluno definir quais devem ser estas  condições de modo a que o sistema tenha um comportamento desejável: imobilize-se depressa e não “derrape” muito. \
b) Modele em lógica temporal linear LT  propriedades que caracterizam o comportamento desejável do sistema. Nomeadamente \
1. ”o veículo imobiliza-se completamente em menos de $t$ segundos”
2. “a velocidade $V$ diminui sempre com o tempo”. 

c) Construa o FOTS que que descreve a discretização do  modelo  que definiu em a. e codifique em SMT’s \
d) Codifique a verificação das propriedades temporais que definiu em b.
## Implementação

----

### Imports

----

In [1]:
from pysmt.shortcuts import *

## Alinea a)

----

<div class="alert alert-block" style="background-color: #EEEEEE; color: #333;">
    <b style="text-align: center; display: block;">
        Autómato Híbrido
    </b>
    <img src="C:\Users\danie\Downloads\drawin.png" alt="Description" width="1400">
</div>



### Alinea c)

----


In [2]:
START = Int(0)
FREE = Int(1) # Não existe força de travagem
STOPPING = Int(2)
BLOCKED = Int(3) # Rodas bloqueadas mas veículo desloca-se
STOPPED = Int(4) # Veículo imobilizado

MODE = {0 : 'START', 1 : 'FREE', 2 : 'STOPPING', 3 : 'BLOCKED', 4 : 'STOPPED'}

a = 0.02 # Atrito
P = 1000 # Peso
Vi = 20 # Velocidade Inicial
T = 0.2 # Max Segundos (Free ou Blocked)

Declararação das variáveis do FOTS associadas ao veículo, onde $t$ representa o tempo, $m$ representa o estado do veículo, $V$ a velocidade do corpo em relação ao solo e $V_r$ a velocidade linear das rodas em relação ao solo.

In [3]:
def declare(i):
    s = {}
    s['t'] = Symbol('t'+str(i), REAL)
    s['m'] = Symbol('m'+str(i), INT)
    s['V'] = Symbol('V'+str(i), REAL)
    s['v'] = Symbol('v'+str(i), REAL)
    s['cr'] = Symbol('cr'+str(i), REAL)
    return s


O estado inicial do FOTS é o seguinte :
\begin{array}{c}
m = \mathsf{START}\wedge v = V_{0}  \wedge V_{0} + 0.1 = V \wedge t = 0 \wedge V_{0}  >= 0 \wedge cr = 0 \\
\end{array}


Iniciamos no estado `Start`, com $V$ = $(V_0)+0.1$. 
Isso é necessário para evitar erros em uma transição temporizada, já que a operação $V-v$ resulta em erro caso $V$ = $v$. 
A variavél $v$ representa a velocidade inicial e tem valor $V_0$, que pela Equação de Fluxo número 7 é o "input" do problema. Além disso, $cr$ é usada para registrar a diferença de tempo. 

In [4]:
def init(s):
    return And(
        Equals(s['m'],START),
        Equals(s['V'],Real(Vi)+0.1),
        s['V'] >= 0.0,
        Equals(Real(Vi),s['v']),
        Equals(s['t'], Real(0)),
        Equals(s['cr'], Real(0))
    )

In [5]:
def trans(curr, prox):
    # Untimed

    startFree = And(
        Equals(curr['m'], START),
        Equals(prox['m'], FREE),
        Equals(curr['t'], prox['t']),
        curr['t'] >= 0,
        prox['t'] >= 0,
        Equals(curr['v'], prox['v']),
        Equals(curr['V'], prox['V']),
        Equals(curr['cr'], prox['cr'])
    )
    freeStopping = And(
        Equals(curr['m'], FREE),
        Equals(prox['m'], STOPPING),
        Equals(curr['v'], prox['v']),
        Equals(prox['cr'], Real(0)),
        Equals(curr['V'], prox['V']),
        Equals(prox['t'], curr['t']),
        Equals(curr['cr'], Real(T)),
        curr['t'] >= 0,
        prox['t'] >= 0
    )
    freeStopping1 = And(
        Equals(curr['m'], FREE),
        Equals(prox['m'], STOPPING),
        Equals(prox['cr'], Real(0)),
        Equals(Real(0), prox['v']),
        Equals(Real(0), prox['V']),
        Equals(prox['t'], curr['t']),
        curr['V'] < 0.52
    )
    stoppingBlocked = And(
        Equals(curr['m'], STOPPING),
        Equals(prox['m'], BLOCKED),
        Equals(curr['v'], prox['v']),
        Equals(curr['t'], prox['t']),
        Equals(prox['cr'], Real(0)),
        Equals(curr['v'], Real(0)),
        Equals(curr['V'], prox['V'])
    )
    blockedFree = And(
        Equals(curr['m'], BLOCKED),
        Equals(prox['m'], FREE),
        Equals(curr['v'], prox['v']),
        Equals(curr['t'], prox['t']),
        Equals(prox['cr'], Real(0)),
        Equals(curr['V'] + 0.1, prox['V']),
        Equals(curr['cr'], Real(T)),
        curr['t'] >= 0,
        prox['t'] >= 0
    )
    stoppingStopped = And(
        Equals(curr['m'], STOPPING),
        Equals(prox['m'], STOPPED),
        Equals(curr['t'], prox['t']),
        Equals(curr['v'], prox['v']),
        Equals(curr['V'], prox['V']),
        Equals(curr['V'], Real(0)),
        Equals(curr['v'], Real(0)),
        Equals(prox['cr'], Real(0))
    )
    blockedStopping = And(
        Equals(curr['m'], BLOCKED),
        Equals(prox['m'], STOPPING),
        Equals(curr['t'], prox['t']),
        curr['V'] < 0.52,
        Equals(prox['cr'], Real(0)),
        (Equals(prox['V'], Real(0))),
        (Equals(prox['v'], Real(0)))
    )

    # Timed - Switchs
    freeFree = And(
        Equals(curr['m'], FREE),
        Equals(prox['m'], FREE),
        curr['t'] < prox['t'],
        curr['V'] > prox['V'],
        curr['v'] > prox['v'],
        prox['v'] >= 0,
        prox['V'] >= 0,
        Equals(Real(T), prox['t'] - curr['t']),
        Equals(curr['cr'], Real(0)),
        Equals(prox['cr'], Real(T)),
        cond(curr['V'], curr['v'], curr['t'], prox['V'], prox['v'], prox['t'], Real(0.1))
    )
    blockedBlocked = And(
        Equals(curr['m'], BLOCKED),
        Equals(prox['m'], BLOCKED),
        Equals(curr['cr'], Real(0)),
        Equals(Real(T),prox['t'] - curr['t']),
        curr['t'] < prox['t'],
        curr['V'] > prox['V'],
        prox['v'] >= 0,
        prox['V'] >= 0,
        Equals(prox['V'], prox['v']),
        Equals(prox['cr'], Real(T)),
        Equals(prox['V'] - curr['V'], (-a * P) * (prox['t'] - curr['t']))
    )
    stoppingStopping = And(
        Equals(curr['m'], STOPPING),
        Equals(prox['m'], STOPPING),
        curr['V'] > prox['V'],
        Equals(prox['v'], Real(0)),
        prox['V'] >= 0,
        curr['v'] > prox['v'],
        curr['t'] < prox['t'],
        cond(curr['V'], curr['v'], curr['t'], prox['V'], prox['v'], prox['t'], Real(2))
    )

    return Or(stoppingStopped, freeStopping1, startFree, freeStopping, freeFree, stoppingStopping, stoppingBlocked,
              blockedBlocked, blockedFree)


def cond(V0, v0, t0, V, v, t, c):
    return And(Equals(V - V0, (-c * (V0 - v0)) * (t - t0)), Equals(v - v0, (-a * P + (c * (V0 - v0))) * (t - t0)))

In [6]:
def gera_traco(declare, init, trans, k):
    with Solver(name="z3") as s:

        trace = [declare(i) for i in range(k)]

        s.add_assertion(init(trace[0]))

        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))

        if s.solve():
            '''
            m = s.get_model()
            for n, v in m:
                print(f'{n} = {v}')
            '''
            for i in range(k):
                print(f'Passo {i}')
                for v in trace[i]:
                    print(f'{v} = {s.get_value(trace[i][v])}')

            print(f'---------')

gera_traco(declare, init, trans, 16)

Passo 0
t = 0.0
m = 0
V = 724178820081175757/36028797018963968
v = 20.0
cr = 0.0
Passo 1
t = 0.0
m = 1
V = 724178820081175757/36028797018963968
v = 20.0
cr = 0.0
Passo 2
t = 3602879701896397/18014398509481984
m = 1
V = 469972158513223789722941500462478146902840015488811/23384026197294446691258957323460528314494920687616
v = 374191187209105730769017421842989987433192700344533/23384026197294446691258957323460528314494920687616
cr = 3602879701896397/18014398509481984
Passo 3
t = 3602879701896397/18014398509481984
m = 2
V = 469972158513223789722941500462478146902840015488811/23384026197294446691258957323460528314494920687616
v = 374191187209105730769017421842989987433192700344533/23384026197294446691258957323460528314494920687616
cr = 0.0
Passo 4
t = 1933912799284716734543476492602016321693881173614243244081072012945/1243527540022323212732066523618931435676649805138605617774218706944
m = 2
V = 22374396738401652293178604557040289069378616889058153149779648952297/2487055080044646425464133047

## 1. ”O veículo imobiliza-se completamente em menos de t segundos”
Sendo T o número de segundos que queremos limitar sendo S o conjunto dos estados .

\begin{array}{c}
(s_{t} >= T \wedge s_{V} = 0) \vee (s_{t} < T)
\end{array}


In [7]:
def termina_t(s, tempo):
    return Or(
        And(
            s['t'] >= tempo,
            Equals(s['V'], Real(0))
        ),
        s['t'] < tempo
    )

def bmc_always1(declare, init, trans, inv, t, K):

    for k in range(1, K + 1):
        with Solver(name="z3") as s:
            trace = [declare(i) for i in range(k)]

            s.add_assertion(init(trace[0]))

            for i in range(k-1):
                s.add_assertion(trans(trace[i], trace[i+1]))

            for i in range(k):
                s.add_assertion(Not(And(inv(trace[i], t))))
            if s.solve():
                for i in range(k):
                    print(f'Passo {i}')
                    for v in trace[i]:
                        print(f'{v} = {s.get_value(trace[i][v])}')
                    print(f'---------')
                print(f'Tem erro')
                return
    print(f'Erro não encontrado')

bmc_always1(declare, init, trans, termina_t, 5, 10)

Erro não encontrado


## 2.“A velocidade V diminui sempre com o tempo”
Sendo S o conjunto dos estados .
\begin{array}{c}
\forall_{i\in length(S)-1}  \quad  se  \quad  s_{i,t} < s_{i+1,t}\quad então \quad s_{i,V} > s_{i+1,V}
\end{array}


In [8]:
p = declare(1)

def veldiminui(c, p):
    return Or(
        And(
            c['t'] < p['t'],
            p['V'] < c['V']
        ),
        Equals(p['t'], c['t'])
    )

def bmc_always2(declare, init, trans, inv, K):

    for k in range(2, K + 1):
        with Solver(name="z3") as s:
            trace = [declare(i) for i in range(k)]
            s.add_assertion(init(trace[0]))

            for i in range(k-1):
                s.add_assertion(trans(trace[i], trace[i+1]))
                s.add_assertion(Not(inv(trace[i], trace[i+1])))

            if s.solve():
                for i in range(k):
                    print(f'Passo {i}')
                    for v in trace[i]:
                        print(f'{v} = {s.get_value(trace[i][v])}')
                    print(f'---------')
                print(f'Tem erro')
                return
    print(f'Erro não encontrado')

bmc_always2(declare, init, trans, veldiminui, 10)


Erro não encontrado
